In [1]:
import os
from types import SimpleNamespace

import dotenv
import openai
import pymongo
import pycmarkgfm
import numpy as np
from IPython import display
from ipywidgets import HTML
from bson import ObjectId

In [2]:
def connect_to_db(collection):
    dotenv.load_dotenv()
    mongodb_uri = os.getenv("MONGODB_URI")
    DBclient = pymongo.MongoClient(mongodb_uri)
    db = DBclient["ML2-Abschlussprojekt"]
    return db[collection]

In [3]:
def get_semester(id):
    collection = connect_to_db("Semester_Data")
    return collection.find_one({"_id":ObjectId(id)})

In [4]:
def get_student(id):
    collection = connect_to_db("Base_Data")
    return collection.find_one({"_id":ObjectId(id)})

In [5]:
def anonymise(student, text):
    if student["gender"].lower()=="männlich":
        aliasname = "[John]"
    else:
        aliasname = "[Jane]"
    alias_text = text.replace(student["firstname"], aliasname)
    
    return alias_text

In [6]:
def format_prompt_content(semester, student):
    allgemeines_lernen = {
        "AktivTeilnehmen": 'aktiv am Unterricht teilnehmen',
        "LeistungZeigen": 'im Unterricht Leistung zeigen',
        "AufmerksamSein": 'während dem Unterricht Aufmerksam sein und den Instruktionen der Lehrperson folgen',
        "SchulinhalteMerken": 'sich schulische Inhalte merken',
        "SchulinhalteAbrufen": 'schulische Inhalte bei Bedarf abrufen',
    }
    assessment = semester["final_assessment"]["allgemeines_lernen"]
    text = "\n".join(
        f" - {student['firstname']} kann {v['assessment']} {allgemeines_lernen[k]} {v['notes']}"
        for k, v in assessment.items()
    )
    return anonymise(student, text)

In [7]:
semester = get_semester("665d8ff86ce65f51c7e17e34")
student = get_student(semester["student_id"])

In [8]:
assessment_prompt = format_prompt_content(semester, student)

In [9]:
system_content = f"""
Du wirst eine Evaluation eines Schülers oder einer Schülerin bekommen.
Die Evaluation wird in kurzen einfachen Sätzen sein.
Dein Task ist es, diese Sätze zu einem Fliesstext zusammenzubringen.
Der Fliesstext soll professionell sein.
Der Fliesstext soll nur Informationen enthalten, die von der Evaluation mitgegeben werden.
Der Fliesstext soll die Schweizer Rechtschreibung nutzen, das heisst, Scharf-S wird durch Doppel-S ersetzt.
"""

In [10]:
first_prompt = f"""
Bitte mache aus der folgenden Einschätzung einen professionellen Fliesstext:
{assessment_prompt}
"""

In [11]:
messages = [
    {
        "role": "system",
        "content": system_content
    },
    {
        "role": "user",
        "content": """
John Doe kann gut aktiv am Unterricht teilnehmen.
John Doe kann gut im schulischen Umfeld Leistung zeigen.
John Doe kann nicht immer während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
John Doe kann teilweise sich schulische Inhalte merken.
John Doe kann noch nicht schulische Inhalte bei Bedarf abrufen.
        """
    },
    {
        "role": "assistant",
        "content": """
John Doe gelingt es gut, aktiv am Unterricht teilzunehmen.
Ausserdem zeigt er meist gute Leistungen im schulischen Umfeld.
Er kann jedoch während des Unterrichts nicht immer Aufmerksam sein und den Instruktionen der Lehrperson folgen.
Ausserdem hat John Doe teilweise Mühe, sich schulische Inhalte zu merken.
Auch gelingt es ihm noch nicht, die gemerkten Inhalte bei Bedarf abzurufen.
        """
    },
    {
        "role": "user",
        "content": """
Jane Doe kann teilweise aktiv am Unterricht teilnehmen.
Jane Doe kann nicht immer im schulischen Umfeld Leistung zeigen.
Jane Doe kann gut während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
Jane Doe kann gut sich schulische Inhalte merken.
Jane Doe kann teilweise schulische Inhalte bei Bedarf abrufen.
        """
    },
    {
        "role": "assistant",
        "content": """
Jane Doe kann nur teilweise aktiv am Unterricht teilnehmen.
Nicht immer gelingt es ihr im schulischen Umfeld Leistungen zu zeigen.
Sie kann jedoch gut während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
Ausserdem merkt sie sich schulische Inhalte gut, kann diese jedoch nur teilweise bei Bedarf abrufen.
        """
    },
    {
        "role": "user",
        "content": first_prompt
    },
]

In [12]:
def get_api_key():
    dotenv.load_dotenv()
    os.getenv("OPENAI_APIKEY")
    return os.getenv("OPENAI_APIKEY")
openai.api_key = get_api_key()

In [13]:
def _openai(messages) -> openai.types.chat.chat_completion.Choice:
    """
    Request is sent to OpenAI
    Return the response in a ChatCompletion-Object
    """
    model_result = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2,
        top_p=1,
    )
    response = model_result.choices[0]
    chain = messages + [response.message]
    return response.message.content, chain

In [14]:
answer, chain = _openai(messages)

In [20]:
def follow_up_prompt(prompt, messages):
    messages.append({"role":"user","content":prompt})
    return messages